In [49]:
import os
import os.path as osp
import torch
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

root_dir = osp.join(os.environ.get("SCRATCH", ""), "ties", "exp_out", "training", "t5-base")
task_to_model_dict = {
    'qasc': osp.join(root_dir, "qasc", "best_model.pt"),
    'quartz': osp.join(root_dir, "quartz", "best_model.pt")
}
tokenizer = AutoTokenizer.from_pretrained('t5-base')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
from src.data.dataset_readers import DATASET_CLASSES
from src.data.Batcher import Batcher
from src.data.PytorchDataset import PytorchDataset
createPytorchDataset_fn = lambda dataset: PytorchDataset(dataset, tokenizer, device)

dataset_kwargs = {
    "few_shot_random_seed": None,
    "num_val_samples": 32,
    "max_datapoints_per_dataset_without_templates": None
}

dataset_reader = DATASET_CLASSES['qasc'](dataset_kwargs)
batcher = Batcher(
    dataset_reader,
    createPytorchDataset_fn,
    train_batchSize=32,
    eval_batchSize=32,
    world_size=1,
    device=0,
)
train_iterator = batcher.get_trainBatches(
    "train", 0
)

In [ ]:
# from src.model.T5Wrapper import T5Wrapper
# transformer = AutoModelForSeq2SeqLM.from_pretrained('t5-base')
# tokenizer = AutoTokenizer.from_pretrained('t5-base')
# model = T5Wrapper(transformer, tokenizer)
# model.to(device)
# model(out)

In [52]:
from tqdm import tqdm
from src.model.T5Wrapper import T5Wrapper

# HPs
N_EPOCHS = 10

# Init model
transformer = AutoModelForSeq2SeqLM.from_pretrained('t5-base')
model = T5Wrapper(transformer, tokenizer)

# Optionally load state_dict
# model.load_state_dict(torch.load(task_to_model_dict['qasc']))
model.to(device)
model.train()
for _ in tqdm(range(N_EPOCHS)):
    out = next(train_iterator)
    model(out)

100%|██████████| 10/10 [00:01<00:00,  8.95it/s]
